In [1]:
#%pip install pandas pyarrow
import pandas as pd

# Load the Parquet file
articles = pd.read_parquet('articles.parquet', engine='pyarrow')
behaviors = pd.read_parquet('train/behaviors.parquet', engine='pyarrow')
history = pd.read_parquet('train/history.parquet', engine='pyarrow')

In [2]:
articles.head()

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,...,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3037230,Ishockey-spiller: Jeg troede jeg skulle dø,ISHOCKEY: Ishockey-spilleren Sebastian Harts h...,2023-06-29 06:20:57,False,Ambitionerne om at komme til USA og spille ish...,2003-08-28 08:55:00,None,article_default,https://ekstrabladet.dk/sport/anden_sport/isho...,...,[],"[Kriminalitet, Kendt, Sport, Katastrofe, Mindr...",142,"[327, 334]",sport,NaN,NaN,NaN,0.9752,Negative
1,3044020,Prins Harry tvunget til dna-test,Hoffet tvang Prins Harry til at tage dna-test ...,2023-06-29 06:21:16,False,Den britiske tabloidavis The Sun fortsætter me...,2005-06-29 08:47:00,"[3097307, 3097197, 3104927]",article_default,https://ekstrabladet.dk/underholdning/udlandke...,...,"[PER, PER]","[Kriminalitet, Kendt, Underholdning, Personfar...",414,[432],underholdning,NaN,NaN,NaN,0.7084,Negative
2,3057622,Rådden kørsel på blå plader,Kan ikke straffes: Udenlandske diplomater i Da...,2023-06-29 06:21:24,False,Slingrende spritkørsel. Grove overtrædelser af...,2005-10-10 07:20:00,[3047102],article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Kriminalitet, Transportmiddel, Bil]",118,[133],nyheder,NaN,NaN,NaN,0.9236,Negative
3,3073151,Mærsk-arvinger i livsfare,FANGET I FLODBØLGEN: Skibsrederens oldebørn må...,2023-06-29 06:21:38,False,To oldebørn af skibsreder Mærsk McKinney Mølle...,2005-01-04 06:59:00,"[3067474, 3067478, 3153705]",article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Erhverv, Privat virksomhed, Livsstil, Familie...",118,[133],nyheder,NaN,NaN,NaN,0.9945,Negative
4,3193383,Skød svigersøn gennem babydyne,44-årig kvinde tiltalt for drab på ekssvigersø...,2023-06-29 06:22:57,False,En 44-årig mormor blev i dag fremstillet i et ...,2003-09-15 15:30:00,None,article_default,https://ekstrabladet.dk/krimi/article3193383.ece,...,[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9966,Negative


In [3]:
behaviors.head()

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0
1,152513,9778745.0,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, 9778623, 9089120, 9778661, ...",[9778661],150224,False,NaN,NaN,NaN,False,298,2.0,48.0
2,155390,NaN,2023-05-24 07:30:33,45.0,NaN,1,"[9778369, 9777856, 9778500, 9778021, 9778627, ...",[9777856],160892,False,NaN,NaN,NaN,False,401,215.0,100.0
3,214679,NaN,2023-05-23 05:25:40,33.0,NaN,2,"[9776715, 9776406, 9776566, 9776071, 9776808, ...",[9776566],1001055,False,NaN,NaN,NaN,False,1357,40.0,47.0
4,214681,NaN,2023-05-23 05:31:54,21.0,NaN,2,"[9775202, 9776855, 9776688, 9771995, 9776583, ...",[9776553],1001055,False,NaN,NaN,NaN,False,1358,5.0,49.0


In [4]:
history.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,13538,"[2023-04-27T10:17:43.000000, 2023-04-27T10:18:...","[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11..."
1,58608,"[2023-04-27T18:48:09.000000, 2023-04-27T18:48:...","[37.0, 61.0, 100.0, 100.0, 55.0, 100.0, 100.0,...","[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,..."
2,95507,"[2023-04-27T15:20:28.000000, 2023-04-27T15:20:...","[60.0, 100.0, 100.0, 21.0, 29.0, 67.0, 49.0, 5...","[9739035, 9738646, 9634967, 9738902, 9735495, ...","[18.0, 29.0, 51.0, 12.0, 10.0, 10.0, 13.0, 24...."
3,106588,"[2023-04-27T08:29:09.000000, 2023-04-27T08:29:...","[24.0, 57.0, 100.0, nan, nan, 100.0, 100.0, 73...","[9738292, 9738216, 9737266, 9737556, 9737657, ...","[9.0, 15.0, 42.0, 9.0, 3.0, 58.0, 26.0, 214.0,..."
4,617963,"[2023-04-27T14:42:25.000000, 2023-04-27T14:43:...","[100.0, 100.0, nan, 46.0, 23.0, 19.0, 61.0, 70...","[9739035, 9739088, 9738902, 9738968, 9738760, ...","[45.0, 29.0, 116.0, 26.0, 34.0, 42.0, 58.0, 59..."


## Create mergeimps.tsv (popularity)
The "mergeimps.tsv'' file needs to store the #click and #impression of each news with time information for building popularity signals. Specifically, we first need to divide the time into several buckets in units of one hour. Thus, since the MIND dataset contains user data in 6 weeks, the time periods can be divided into 2476 buckets. Then, we count the #impressions and #clicks of a news in each bucket and store the results in each line of ``mergeimps.tsv'', where the format is :

[News_ID] TAB [Bucket_ID] TAB #Cliks TAB #Impressions.
 
In MIND, since we did not obtain the click information for test data, which can substantially affect the estimation of popularity signals. Thus, I advise that you can use the validation dataset of MIND for evaluation.

In [5]:
behaviors[:10]

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0
1,152513,9778745.0,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, 9778623, 9089120, 9778661, ...",[9778661],150224,False,NaN,NaN,NaN,False,298,2.0,48.0
2,155390,NaN,2023-05-24 07:30:33,45.0,NaN,1,"[9778369, 9777856, 9778500, 9778021, 9778627, ...",[9777856],160892,False,NaN,NaN,NaN,False,401,215.0,100.0
3,214679,NaN,2023-05-23 05:25:40,33.0,NaN,2,"[9776715, 9776406, 9776566, 9776071, 9776808, ...",[9776566],1001055,False,NaN,NaN,NaN,False,1357,40.0,47.0
4,214681,NaN,2023-05-23 05:31:54,21.0,NaN,2,"[9775202, 9776855, 9776688, 9771995, 9776583, ...",[9776553],1001055,False,NaN,NaN,NaN,False,1358,5.0,49.0
5,214684,NaN,2023-05-23 05:32:21,10.0,NaN,2,"[9776508, 9767490, 9776049, 9776544, 9776551, ...",[9776508],1001055,False,NaN,NaN,NaN,False,1358,52.0,100.0
6,214691,NaN,2023-05-23 05:30:46,18.0,NaN,2,"[9759955, 9776449, 9775804, 9776369, 9488213, ...",[9776691],1001055,False,NaN,NaN,NaN,False,1358,4.0,37.0
7,369958,NaN,2023-05-24 14:25:56,16.0,NaN,2,"[9776023, 9778158, 9776929, 9527795, 7594265]",[9778158],1469458,False,NaN,NaN,NaN,False,1623,0.0,NaN
8,369959,NaN,2023-05-24 14:23:14,161.0,NaN,2,"[9779186, 9779289, 9777397, 9779185, 9778813, ...",[9779071],1469458,False,NaN,NaN,NaN,False,1623,16.0,NaN
9,370414,NaN,2023-05-24 14:48:54,9.0,NaN,2,"[9779408, 9779377, 9779289, 9778945, 9777182, ...",[9777182],1470585,False,NaN,NaN,NaN,False,1678,13.0,41.0


In [6]:
# Assuming 'impression_time' is already in datetime format, if not convert it
behaviors['impression_time'] = pd.to_datetime(behaviors['impression_time'])

# Set the start date of your data collection period
start_date = pd.to_datetime('2023-05-18')

# Calculate the hour buckets since the start of the period
behaviors['bucket_id'] = ((behaviors['impression_time'] - start_date).dt.total_seconds() // 3600).astype(int)

behaviors[["bucket_id", "impression_time", "article_ids_inview", "article_ids_clicked"]].head()

,bucket_id,impression_time,article_ids_inview,article_ids_clicked
0,93,2023-05-21 21:06:50,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966]
1,151,2023-05-24 07:31:26,"[9778669, 9778736, 9778623, 9089120, 9778661, ...",[9778661]
2,151,2023-05-24 07:30:33,"[9778369, 9777856, 9778500, 9778021, 9778627, ...",[9777856]
3,125,2023-05-23 05:25:40,"[9776715, 9776406, 9776566, 9776071, 9776808, ...",[9776566]
4,125,2023-05-23 05:31:54,"[9775202, 9776855, 9776688, 9771995, 9776583, ...",[9776553]


In [21]:
double_dict = {}

for i, row in behaviors.iterrows():

    for article_id in row["article_ids_inview"]:
        if str(article_id) + "-" + str(row["bucket_id"]) not in double_dict:
            double_dict[str(article_id) + "-" + str(row["bucket_id"])] = {"article_id": article_id, "bucket_id": row["bucket_id"], "clicks": 0, "impressions": 0}
        double_dict[str(article_id) + "-" + str(row["bucket_id"])]["impressions"] += 1
            
    for article_id in row["article_ids_clicked"]:
        if str(article_id) + "-" + str(row["bucket_id"]) not in double_dict:
            double_dict[str(article_id) + "-" + str(row["bucket_id"])] = {"article_id": article_id, "bucket_id": row["bucket_id"], "clicks": 0, "impressions": 0}
        double_dict[str(article_id) + "-" + str(row["bucket_id"])]["clicks"] += 1
        
mergeimps = pd.DataFrame(double_dict.values())
mergeimps

,article_id,bucket_id,clicks,impressions
0,9774516,93,1,34
1,9771051,93,4,52
2,9770028,93,1,42
3,9775402,93,6,52
4,9774461,93,12,46
...,...,...,...,...
28817,9688894,104,0,1
28818,9657528,10,0,1
28819,9769356,10,0,1
28820,9746428,10,0,1


In [20]:
# mergeimps.to_excel("mergeimps.xlsx", index=False)
mergeimps.to_csv("mergeimps.tsv", sep="\t", index=False, header=False)

In [16]:
# tests
test = mergeimps.groupby("article_id").agg({"clicks": "sum", "impressions": "sum", "bucket_id" :["nunique", "count"]}).reset_index()
test

article_id clicks impressions bucket_id      
                   sum         sum   nunique count
0       4721905      0           5         2     2
1       5345489      0           1         1     1
2       5986757      0          32        16    16
3       6141159      0           1         1     1
4       6141222      0           6         3     3
...         ...    ...         ...       ...   ...
2473    9780162      0           4         3     3
2474    9780181     14         123         2     2
2475    9780193      6         102         2     2
2476    9780195      4          89         2     2
2477    9780271      0          10         1     1

[2478 rows x 5 columns]

## Create V21UrlDocs22_title_json.tsv (necessary?)
For implementing our method based on MIND, the "V21UrlDocs22_title_json.tsv'' needs to store all news in the training/val/test set. Each line stores the information of a single news article, where format is based on 

[News_ID] TAB [Category] TAB [Subcategory] TAB [Title] TAB [Url] TAB [Abstract] TAB [Body].
 
You can modify my codes to avoid the load of some unnecessary news data (e.g., Url, Abstract, and Body)

## Create train/val/test.tsv